<a href="https://colab.research.google.com/github/Thakurvighnesh/HamoMLDatascience/blob/main/Logistic_Regression_tutorials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the libraries
import pandas as pd
import numpy as np

In [4]:
#using pandas to download the dataset (uncomment the code below and execute them)
df = pd.read_csv('https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae')
df.to_csv('dataset.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
#load the dataset
df = pd.read_csv('dataset.csv', low_memory=False)
df.head()

,Unnamed: 0,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [6]:

#drop the unnamed column

df.drop('Unnamed: 0', axis = 1, inplace = True)

In [7]:
#check distribution of target variable
df['QScore'].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [8]:

#checking for null values in the dataset
df.isna().sum()

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [9]:
#for simplicity, we will drop the rows with missing values.
df.dropna(inplace = True)
df.isna().sum()
#An obvious change in our target variable after removing the missing values is that there are only three classes left 
#and from the distribution of the 3 classes, we can see that there is an obvious imbalance between the classes. 
#There are methods that can be applied to handle this imbalance such as oversampling and undersampling.
#Oversampling involves increasing the number of instances in the class with fewer instances while undersampling 
#involves reducing the data points in the class with more instances.

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [10]:
#reset the dataframe index
df = df.reset_index(drop = True)

In [11]:
#An   obvious   change   in   our   target   variable   after   removing   the   missing   values   is   that   there  are   only   three   classes   left  
#and   from   the   distribution   of   the   3   classes,   we   can   see   that  there   is   an   obvious   imbalance   between   the   classes. 
#There   are   methods   that   can   be   applied   to  handle   this   imbalance   such   as   oversampling   and   undersampling.  
#Oversampling   involves   increasing   the   number   of   instances   in   the   class   with   fewer   instances  while   undersampling   
#involves   reducing   the   data   points   in   the   class   with   more   instances.  

#For now, we will convert this to a binary classification problem by combining class '2A' and '1A'.

df['QScore'] = df['QScore'].replace(['1A'], '2A')
df.QScore.value_counts()

3A    51473
2A      240
Name: QScore, dtype: int64

In [12]:
#separating the target variable and 
#selecting some samples
df_2A = df[df.QScore=='2A']
df_3A = df[df.QScore=='3A'].sample(350)
data_df = df_2A.append(df_3A)
data_df

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
1096,Algeria,2016,4,AreaPerCap,2.072989e-01,8.112722e-01,0.048357265,2.258528e-02,2.998367e-02,0.000000e+00,1.119497e+00,2A
1097,Algeria,2016,4,AreaTotHA,8.417600e+06,3.294260e+07,1963600,9.171000e+05,1.217520e+06,0.000000e+00,4.545842e+07,2A
1098,Algeria,2016,4,BiocapPerCap,2.021916e-01,2.636077e-01,0.027166736,7.947991e-03,2.924496e-02,0.000000e+00,5.301590e-01,2A
1099,Algeria,2016,4,BiocapTotGHA,8.210214e+06,1.070408e+07,1103135.245,3.227369e+05,1.187524e+06,0.000000e+00,2.152769e+07,2A
1100,Algeria,2016,4,EFConsPerCap,6.280528e-01,1.810332e-01,0.162800822,1.472910e-02,2.924496e-02,1.391455e+00,2.407316e+00,2A
...,...,...,...,...,...,...,...,...,...,...,...,...
983,Albania,2002,3,EFProdTotGHA,8.274787e+05,5.697089e+05,147121.7536,2.547538e+04,6.407975e+04,1.353172e+06,2.987037e+06,3A
4642,Bhutan,2012,18,BiocapPerCap,2.993287e-01,3.362938e-01,4.311682093,1.343913e-02,2.618047e-01,0.000000e+00,5.222548e+00,3A
46757,Burkina Faso,1980,233,EFConsPerCap,3.224507e-01,3.752669e-01,0.480790295,3.433654e-03,3.187197e-02,4.805944e-02,1.261873e+00,3A
31262,Nigeria,2000,159,EFConsTotGHA,7.482893e+07,1.075780e+07,30676452.16,4.792689e+06,5.074922e+06,1.867414e+07,1.448049e+08,3A


In [13]:
#So proceeding from line 30
#we would like to reshuffle the target variable so that it will be well distributed.
#so we import the scikit-learn utils class, calling the shuffle function, and pass the data frame into the parenthesis.

#After which we reset index calling the drop function, so as to have one index column for the dataset

#to reshuffle the dataset for randomness
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.QScore.value_counts()

3A    350
2A    240
Name: QScore, dtype: int64

In [14]:
#check the datatype of the dataset
data_df.dtypes


country            object
year                int64
country_code        int64
record             object
crop_land         float64
grazing_land      float64
forest_land        object
fishing_ground    float64
built_up_land     float64
carbon            float64
total             float64
QScore             object
dtype: object

In [ ]:
#let's preview the feature to be encoded before encoding
data_df.record

In [15]:
# from the above, we will be dropping country code, country and year
#because they are ambiguous to what we want to predict
#and also we will encode the record feature

#One of the feature is categorical, so we need to encode it ahead 
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data_df.record = encoder.fit_transform(data_df.record)

In [16]:
#let's preview the encoded feature
data_df.record

0      3
1      3
2      2
3      0
4      5
      ..
585    0
586    5
587    2
588    2
589    6
Name: record, Length: 590, dtype: int64

In [17]:
#more preprocessing
data_df = data_df.drop(columns=['country_code', 'country', 'year'])
X = data_df.drop(columns = 'QScore')
y = data_df['QScore']

In [18]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
y_train.value_counts()

3A    245
2A    168
Name: QScore, dtype: int64

In [24]:
#installing imblearn module (uncomment the pip command to install imblearn)
#!pip install imblearn

#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)

#the encoded feature
#encode   categorical   variable  
#why the label encoder was used to transform the 'record' column is what i don't understand...i thought that it was supposed to be a categorical variable???
#->The column label was transformed from categorical data which are strings, to non categorical data which are numbers,
# because the SMOTE library does not accept strings only numbers. The alternative would have been to drop the 'record' column entirely,
# but since it's very important in the dataset we had to transform it so the SMOTE library could work on the dataset.

#SMOTE (synthetic minority oversampling technique) is one of the most commonly used oversampling methods to solve the imbalance problem.
#It aims to balance class distribution by randomly increasing minority class examples by replicating them. x_train.record

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [25]:


#There is still an imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this.
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)
y_train.value_counts()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


3A    245
2A    168
Name: QScore, dtype: int64

In [31]:
#min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_balanced = pd.DataFrame(x_train_balanced, columns=x_train.columns)
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train_balanced.drop(columns=['record']).columns)
normalised_train_df['record'] = x_train_balanced['record']


x_test = x_test.reset_index(drop=True)
normalised_test_df = scaler.transform(x_test.drop(columns=['record']))
normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_test.drop(columns=['record']).columns)
normalised_test_df['record'] = x_test['record']

In [28]:
#check x_test
x_test

,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total
0,5,4.158552e+06,5.137046e+05,2811412.039,1.395335e+05,5.379828e+05,9.234231e+05,9.084608e+06
1,7,7.632476e+06,1.112202e+07,5138340.002,4.817236e+04,4.815138e+05,1.260642e+06,2.568316e+07
2,1,1.820000e+04,1.390000e+04,8128400,4.602000e+06,1.013870e+03,0.000000e+00,1.276351e+07
3,6,2.525098e-01,3.435519e-01,0.817085801,7.470000e-05,7.062765e-02,2.993072e-01,1.783157e+00
4,3,9.221366e+03,4.706845e+02,444785.889,3.197453e+06,4.893431e+03,0.000000e+00,3.656824e+06
...,...,...,...,...,...,...,...,...
172,4,3.157451e-01,3.718924e-01,0.665547024,1.756159e-02,7.062765e-02,4.170737e-01,1.858447e+00
173,1,4.924728e+06,2.751561e+06,4008129.34,6.897252e+06,2.927351e+05,0.000000e+00,1.887441e+07
174,0,2.727179e-01,1.081437e+00,0.922864406,5.819276e-01,2.779202e-02,0.000000e+00,2.886739e+00
175,2,4.456409e-01,4.409817e+00,3.248838238,8.150677e-01,4.779697e-02,0.000000e+00,8.967161e+00


In [32]:
#check y_test
y_test

225    3A
14     3A
85     2A
418    2A
132    3A
       ..
346    2A
369    3A
140    3A
533    3A
171    2A
Name: QScore, Length: 177, dtype: object

In [29]:
#check x_train
x_train

,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total
285,2,2.993287e-01,3.362938e-01,4.311682093,1.343913e-02,0.261805,0.000000,5.222548e+00
113,0,2.085225e-01,6.528357e-01,4.227857177,4.443133e-02,0.130795,0.000000,5.264442e+00
18,4,5.363930e-01,3.948624e-01,0.152293385,3.584621e-01,0.182896,4.338376,5.963282e+00
76,6,2.915519e+00,3.420547e+00,1.059972742,7.468023e-02,0.065155,6.675360,1.421123e+01
206,0,5.016420e-01,8.353072e-02,0.073346837,2.036776e-02,0.029992,0.000000,7.088794e-01
...,...,...,...,...,...,...,...,...
277,6,1.238076e-01,1.683009e-02,0.00232796,1.255043e-03,0.028138,1.323674,1.496033e+00
9,4,3.395404e-01,7.870693e-02,0.245210175,1.218363e-03,0.077468,0.113645,8.557883e-01
359,0,2.218488e-01,6.060860e-02,0.13036614,7.585633e-02,0.035484,0.000000,5.241638e-01
192,3,7.835674e+03,4.690748e+02,444667.6706,3.186518e+06,4065.139036,0.000000,3.643556e+06


In [30]:
#check y_train
y_train

285    3A
113    3A
18     3A
76     3A
206    3A
       ..
277    2A
9      3A
359    3A
192    3A
559    3A
Name: QScore, Length: 413, dtype: object

In [33]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)
#returns
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


# LESSON 2
# MEASURING CLASSIFICATION PERFORMANCE

Cross Validation and Accuracy